In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns
sns.set(style='dark')
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#Probably can`t be finished because of huge amount of data with kaggle hardware, add nrows parameter to run here
#Load Data
df1=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")#,nrows = 50000
df2=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
# df4=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
# df7=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
# df8=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")

In [ ]:
print (df1[' Label'].unique())
print (df2[' Label'].unique())
print (df3[' Label'].unique())
# print (df4[' Label'].unique())
print (df5[' Label'].unique())
print (df6[' Label'].unique())


In [ ]:
df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
# df = pd.concat([df,df4])
# del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
# df = pd.concat([df,df7])
# del df7
# df = pd.concat([df,df8])
# del df8

In [ ]:
data = df.copy()

In [ ]:
data.info()

In [ ]:
data.head(20)

In [ ]:
deleteCol = []
for column in data.columns:
    if data[column].isnull().values.any():
        deleteCol.append(column)
for column in deleteCol:
    data.drop([column],axis=1,inplace=True)

In [ ]:
deleteCol = []
for column in data.columns:
    if column == ' Label':
        continue
    elif data[column].dtype==np.object:
        deleteCol.append(column)
for column in deleteCol:
    data.drop(column,axis=1,inplace=True)

In [ ]:
data[' Flow Duration'].unique()

In [ ]:
for column in data.columns:
    if data[column].dtype == np.int64:
        maxVal = data[column].max()
        if maxVal < 120:
            data[column] = data[column].astype(np.int8)
        elif maxVal < 32767:
            data[column] = data[column].astype(np.int16)
        else:
            data[column] = data[column].astype(np.int32)
            
    if data[column].dtype == np.float64:
        maxVal = data[column].max()
        minVal = data[data[column]>0][column]
        if maxVal < 120 and minVal>0.01 :
            data[column] = data[column].astype(np.float16)
        else:
            data[column] = data[column].astype(np.float32)
            

In [ ]:
data.info()

In [ ]:
y = data[' Label']
X = data.drop([' Label'],axis=1)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
bestfeatures = SelectKBest(score_func=f_classif, k=10)
fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(30,'Score'))  #print 10 best features

In [ ]:
feature = pd.DataFrame()
n = len(featureScores['Specs'])
for i in featureScores.nlargest(n//2,'Score')['Specs']:
        feature[i] = data[i]
feature[' Label'] = data[' Label']

In [ ]:
feature.info()

In [ ]:
from matplotlib import pyplot as plt 
import seaborn as sns
fig= plt.figure(figsize=(40,40))
sns.heatmap(feature.corr(), annot=True)

Clearly there is an exellent correlation in - 

 Bwd Packet Length Std     
 Bwd Packet Length Mean    
 Avg Bwd Segment Size      
Bwd Packet Length Max    
 Packet Length Std         
 Average Packet Size       
 Packet Length Mean        
 Max Packet Length       
 Packet Length Variance   
 
 
Also,correlattion lies in - 

 Flow IAT Max            
 Idle Max                
 Fwd IAT Max             
 Flow IAT Std              
 Idle Std                  
Idle Mean                  

In [ ]:
feature.drop([' Bwd Packet Length Mean'],axis=1,inplace=True)
feature.drop([' Avg Bwd Segment Size'],axis=1,inplace=True)
feature.drop(['Bwd Packet Length Max'],axis=1,inplace=True)
feature.drop([' Packet Length Std'],axis=1,inplace=True)
feature.drop([' Average Packet Size'],axis=1,inplace=True)
feature.drop([' Packet Length Mean'],axis=1,inplace=True)
feature.drop([' Max Packet Length'],axis=1,inplace=True)
feature.drop([' Packet Length Variance'],axis=1,inplace=True)


feature.drop([' Idle Max'],axis=1,inplace=True)
feature.drop([' Fwd IAT Max'],axis=1,inplace=True)
feature.drop([' Flow IAT Std'],axis=1,inplace=True)
feature.drop([' Idle Std'],axis=1,inplace=True)
feature.drop(['Idle Mean'],axis=1,inplace=True)

In [ ]:
fig= plt.figure(figsize=(40,40))
sns.heatmap(feature.corr(), annot=True)

In [ ]:
feature[' Label'].value_counts()

In [ ]:
attackType = feature[' Label'].unique()
feature[' Label'] = feature[' Label'].astype('category')
feature[' Label'] = feature[' Label'].astype("category").cat.codes

In [ ]:
print (attackType)
print (feature[' Label'].value_counts())

In [ ]:
feature.info()

In [ ]:
feature0 = feature[feature[' Label'] == 0]
feature0.info()

In [ ]:
sns.boxplot(x = feature0[' Bwd Packet Length Std'])

In [ ]:
sns.boxplot(x = feature0[' Bwd Packet Length Min'])

In [ ]:
sns.boxplot(x = feature0[' Min Packet Length'])

In [ ]:
sns.boxplot(x = feature0[' Fwd IAT Std'])

In [ ]:
sns.boxplot(x = feature0['Init_Win_bytes_forward'])

In [ ]:
sns.boxplot(x = feature0[' Flow IAT Max'])

In [ ]:
sns.boxplot(x = feature0[' Bwd Packets/s'])

In [ ]:
sns.boxplot(x = feature0['Bwd IAT Total'])

The following features have outliers 

' Bwd Packet Length Std'
 
' Bwd Packet Length Min'
 
' Min Packet Length'
 
' Bwd Packets/s'

In [ ]:
cols = [' Bwd Packet Length Std',
        ' Bwd Packet Length Min',
        ' Min Packet Length',
        ' Bwd Packets/s']

for col in cols:
    Q1 = feature0[col].quantile(0.25)
    Q3 = feature0[col].quantile(0.75)
    IQR = Q3 - Q1
    print(IQR)

In [ ]:
sns.distplot(feature0[' Bwd Packets/s'], kde=False,color="b")

In [ ]:
sns.distplot(feature0[' Min Packet Length'], kde=False,color="b")

In [ ]:
sns.distplot(feature0[' Bwd Packet Length Min'], kde=False,color="b")

In [ ]:
sns.distplot(feature0[' Bwd Packet Length Std'], kde=False,color="b")

In [ ]:
y = feature[' Label']
X = feature.drop([' Label'],axis=1)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler('majority')
X_rus, y_rus = rus.fit_sample(X, y)

# print('Removed indexes:', id_rus)

# plot_2d_space(X_rus, y_rus, 'Random under-sampling')

In [ ]:
# from imblearn.combine import SMOTEENN
# sme = SMOTEENN(random_state=42)
# X_res, y_res = sme.fit_sample(X, y)

In [ ]:
X_rus.info()

In [ ]:
y_rus.value_counts()

In [ ]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek('not majority')
X_smt, y_smt = smt.fit_sample(X_rus, y_rus)

In [ ]:
y_smt.value_counts()

In [ ]:
#Split dataset on train and test
from sklearn.model_selection import train_test_split
train_X, test_X,train_y,test_y=train_test_split(X_smt,y_smt,test_size=0.3, random_state=10)

#Exploratory Analysis
# Descriptive statistics
# print (train_X.describe())


# Packet Attack Distribution
# train[' Label'].value_counts()
# test[' Label'].value_counts()

In [ ]:
#Scalling numerical attributes
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# extract numerical attributes and scale it to have zero mean and unit variance  
cols = train_X.select_dtypes(include=['float32','float16','int32','int16','int8']).columns
sc_train = scaler.fit_transform(train_X.select_dtypes(include=['float32','float16','int32','int16','int8']))
sc_test = scaler.fit_transform(test_X.select_dtypes(include=['float32','float16','int32','int16','int8']))

# turn the result back to a dataframe
train_X = pd.DataFrame(sc_train, columns = cols)
test_X = pd.DataFrame(sc_test, columns = cols)

In [ ]:
train_X.head()

In [ ]:
# #Feature Selection
# from sklearn.ensemble import ExtraTreesClassifier
# etc = ExtraTreesClassifier();

# # fit random forest classifier on the training set
# etc.fit(train_X, train_y);

In [ ]:
# # extract important features
# score = np.round(etc.feature_importances_,3)
# importances = pd.DataFrame({'feature':train_X.columns,'importance':score})
# importances = importances.sort_values('importance',ascending=False).set_index('feature')

# # plot importances
# plt.rcParams['figure.figsize'] = (11, 4)
# importances.plot.bar();

In [ ]:
#Dataset Partition
X_train,X_test,Y_train,Y_test = train_test_split(train_X,train_y,train_size=0.70, random_state=2)

#Fitting Models
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
# Train KNeighborsClassifier Model
# KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
# KNN_Classifier.fit(X_train, Y_train); 

# Random Forest Model 
# RTC_Classifier = RandomForestClassifier(criterion='entropy')
# RTC_Classifier.fit(X_train, Y_train)

# Train LogisticRegression Model
# LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=0)
# LGR_Classifier.fit(X_train, Y_train);

# Train Gaussian Naive Baye Model
# BNB_Classifier = BernoulliNB()
# BNB_Classifier.fit(X_train, Y_train)

# Train Decision Tree Model
DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
DTC_Classifier.fit(X_train, Y_train)
print ('DTC Classifier Trained')

# XGB_Classifier = xgb.XGBClassifier(criterion='entropy', random_state=0)
# XGB_Classifier.fit(X_train, Y_train)
# print ('XGB Classifier Trained')




# ADA_Classifier = AdaBoostClassifier(
#     DTC_Classifier,
#     n_estimators=100,
#     learning_rate=1.5)

# ADA_Classifier.fit(X_train, Y_train)
# print ('ADA Classifier Trained')



In [ ]:
#Evaluate Models
from sklearn import metrics

models = []

models.append(('Decision Tree Classifier', DTC_Classifier))
# models.append(('XGB Classifier', XGB_Classifier))
# models.append(('Random Forest Classifier', RTC_Classifier))
# models.append(('LogisticRegression', LGR_Classifier))

for i, v in models:
    vpred = v.predict(X_train)
    scores = cross_val_score(v, X_train, Y_train, cv=10)
    accuracy = metrics.accuracy_score(Y_train, vpred)
    confusion_matrix = metrics.confusion_matrix(Y_train, vpred)
    classification = metrics.classification_report(Y_train, vpred)
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()

In [ ]:
#Validate Models
for i, v in models:
    accuracy = metrics.accuracy_score(Y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(Y_test, v.predict(X_test))
    classification = metrics.classification_report(Y_test, v.predict(X_test))
    print()
    print('============================== {} Model Test Results =============================='.format(i))
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()        